In [1]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.python.platform import gfile
import os.path
import re
import sys
import tarfile
from subprocess import Popen, PIPE, STDOUT

def run(cmd):
    p = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True)
    return p.stdout.read()


In [2]:
# All the constants to run this notebook.

model_dir = '/tmp/imagenet'
image_file = ""
num_top_predictions = 5
DATA_URL = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'

IMG_URL = 'hdfs://localhost:9000/data/im10*.jpg'
TAG_URL = 'hdfs://localhost:9000/data/meta/tags/tags10*.txt'


In [12]:
# list of most used tags ordered from the most to the less used
most_used_tags = sc.binaryFiles(TAG_URL).flatMap(lambda x:x[1].splitlines()).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[1],ascending=False).take(1000)

def clean_img_rdd(x):
    key = os.path.basename(x[0]).split('.')[0][2:]    
    return (key,x[1])

def clean_tags_rdd(x):
    key = os.path.basename(x[0]).split('.')[0][4:]  
    tags = x[1].splitlines()
    value = list()
    for tag in tags:
        for pos, val in enumerate(most_used_tags):
            if val[0] == tag:
                print("ok!")
                value.append(pos)    
                break
    return (key,value)
    
def read_file_index():
    im = sc.binaryFiles(IMG_URL).map(clean_img_rdd)
    tg = sc.binaryFiles(TAG_URL).map(clean_tags_rdd)
    
    return im.join(tg)


In [13]:
def maybe_download_and_extract():
    """Download and extract model tar file."""
    from six.moves import urllib
    dest_directory = model_dir
    if not os.path.exists(dest_directory):
        os.makedirs(dest_directory)
    filename = DATA_URL.split('/')[-1]
    filepath = os.path.join(dest_directory, filename)
    if not os.path.exists(filepath):
        filepath2, _ = urllib.request.urlretrieve(DATA_URL, filepath)
        print("filepath2", filepath2)
        statinfo = os.stat(filepath)
        print('Succesfully downloaded', filename, statinfo.st_size, 'bytes.')
        tarfile.open(filepath, 'r:gz').extractall(dest_directory)
    else:
        print('Data already downloaded:', filepath, os.stat(filepath))

maybe_download_and_extract()


('Data already downloaded:', '/tmp/imagenet/inception-2015-12-05.tgz', posix.stat_result(st_mode=33188, st_ino=1975180, st_dev=66309, st_nlink=1, st_uid=0, st_gid=0, st_size=88931400, st_atime=1515549575, st_mtime=1515549584, st_ctime=1515549584))


In [14]:
image_data = read_file_index()

In [15]:
label_lookup_path = os.path.join(model_dir, 'imagenet_2012_challenge_label_map_proto.pbtxt')
uid_lookup_path = os.path.join(model_dir, 'imagenet_synset_to_human_label_map.txt')

def load_lookup():
    """Loads a human readable English name for each softmax node.
    
    Args:
        label_lookup_path: string UID to integer node ID.
        uid_lookup_path: string UID to human-readable string.

    Returns:
        dict from integer node ID to human-readable string.
    """
    if not gfile.Exists(uid_lookup_path):
        tf.logging.fatal('File does not exist %s', uid_lookup_path)
    if not gfile.Exists(label_lookup_path):
        tf.logging.fatal('File does not exist %s', label_lookup_path)

    # Loads mapping from string UID to human-readable string
    proto_as_ascii_lines = gfile.GFile(uid_lookup_path).readlines()
    uid_to_human = {}
    p = re.compile(r'[n\d]*[ \S,]*')
    for line in proto_as_ascii_lines:
        parsed_items = p.findall(line)
        uid = parsed_items[0]
        human_string = parsed_items[2]
        uid_to_human[uid] = human_string

    # Loads mapping from string UID to integer node ID.
    node_id_to_uid = {}
    proto_as_ascii = gfile.GFile(label_lookup_path).readlines()
    for line in proto_as_ascii:
        if line.startswith('  target_class:'):
            target_class = int(line.split(': ')[1])
        if line.startswith('  target_class_string:'):
            target_class_string = line.split(': ')[1]
            node_id_to_uid[target_class] = target_class_string[1:-2]

    # Loads the final mapping of integer node ID to human-readable string
    node_id_to_name = {}
    for key, val in node_id_to_uid.items():
        if val not in uid_to_human:
            tf.logging.fatal('Failed to locate: %s', val)
        name = uid_to_human[val]
        node_id_to_name[key] = name

    return node_id_to_name

node_lookup = load_lookup()

node_lookup_bc = sc.broadcast(node_lookup)

In [16]:
model_path = os.path.join(model_dir, 'classify_image_graph_def.pb')
with gfile.FastGFile(model_path, 'rb') as f:
    model_data = f.read()
    
model_data_bc = sc.broadcast(model_data)


In [17]:
def run_image(sess, img_id, image, tags, node_lookup):

    scores = []
    
    softmax_tensor = sess.graph.get_tensor_by_name('softmax:0')
    predictions = sess.run(softmax_tensor,
                            {'DecodeJpeg/contents:0': image})
    predictions = np.squeeze(predictions)
    top_k = predictions.argsort()[-num_top_predictions:][::-1]
    scores = []
    for node_id in top_k:
        if node_id not in node_lookup:
            human_string = ''
        else:
            human_string = node_lookup[node_id]
        score = predictions[node_id]
        #scores.append((human_string, score))
        scores.append((node_id, score))
    return (tags, scores, img_id)

def apply_inference(image_entry):
    img_id = image_entry[0]
    image = image_entry[1][0]
    tags = image_entry[1][1]
    with tf.Graph().as_default() as g:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(model_data_bc.value)
        tf.import_graph_def(graph_def, name='')
        with tf.Session() as sess:
            labelled = run_image(sess, img_id, image, tags, node_lookup_bc.value)
            return labelled


In [18]:
# filter the images without tags -> x[1][1] are tags
# apply inference in images
inference_images = image_data.filter(lambda x: x[1][1]).map(apply_inference)

In [19]:
local_inference_images = inference_images.collect()

local_inference_images

[([187, 126, 621, 718, 523, 155, 400],
  [(761, 0.22730482),
   (819, 0.049915362),
   (382, 0.03734066),
   (958, 0.034226477),
   (336, 0.030088237)],
  u'1087'),
 ([857, 87, 402, 225, 761],
  [(57, 0.56658959),
   (12, 0.35133642),
   (155, 0.0081336638),
   (65, 0.00391897),
   (136, 0.0036721569)],
  u'1085'),
 ([144, 711, 871, 505, 586, 238],
  [(648, 0.6447764),
   (453, 0.033781823),
   (365, 0.033299211),
   (838, 0.008641962),
   (327, 0.0059704361)],
  u'1081'),
 ([832, 7, 215, 363],
  [(581, 0.054643136),
   (992, 0.042760666),
   (356, 0.035340082),
   (895, 0.031024935),
   (535, 0.028813684)],
  u'1029'),
 ([703, 686, 253, 11, 570, 54, 799, 898],
  [(967, 0.78006184),
   (942, 0.013801795),
   (992, 0.0097311018),
   (330, 0.0086515183),
   (478, 0.0053733536)],
  u'1089'),
 ([100,
   45,
   449,
   331,
   840,
   468,
   129,
   387,
   18,
   20,
   46,
   75,
   787,
   353,
   95,
   654,
   296,
   2,
   13,
   7,
   841],
  [(985, 0.68836701),
   (945, 0.14225172)

In [46]:
def merge_tag_as_label(categories_and_tags):
    tags = categories_and_tags[0]
    categories = categories_and_tags[1]
    paired = []
    for tag in tags:
        for category in categories:
            paired.append((tag,category))
    return paired

def merge_inference_as_label(categories_and_tags):
    tags = categories_and_tags[0]
    categories = categories_and_tags[1]
    paired = []
    for category in categories:
        for tag in tags:
            paired.append((category[0],(tag,category[1])))
    return paired


#tag_as_label = inference_images.flatMap(merge_tag_as_label).aggregateByKey((),(lambda x,y: x+(y,)),(lambda x,y: x+(y,))).collect()
#inference_as_label = inference_images.flatMap(merge_inference_as_label).aggregateByKey((),(lambda x,y: x+(y,)),(lambda x,y: x+(y,))).collect()


In [47]:
# merge images and tags
# filters inferences with more than 50% of probability
inference_as_label = inference_images.flatMap(merge_inference_as_label).filter(lambda x:x[1][1]>0.5).aggregateByKey(list(),(lambda x,y: x+list((y,))),(lambda i,j: i+j)).filter(lambda x:len(x[1])>1).collect()

inference_as_label

[(596,
  [('louisville', 0.72243607),
   ('kentucky', 0.72243607),
   ('skyline', 0.72243607),
   ('indiana', 0.72243607),
   ('fallsoftheohio', 0.72243607),
   ('1855mm', 0.72243607)]),
 (944,
  [('washington', 0.89433146),
   ('wa', 0.89433146),
   ('spaceneedle', 0.89433146),
   ('skyline', 0.89433146),
   ('seattle', 0.89433146),
   ('panorama', 0.89433146),
   ('elliottbay', 0.89433146),
   ('clouds', 0.89433146),
   ('city', 0.89433146)]),
 (330,
  [('northside', 0.62856919),
   ('pittsburgh', 0.62856919),
   ('eastallegheny', 0.62856919),
   ('sky', 0.62856919),
   ('clouds', 0.62856919)]),
 (722,
  [('1776', 0.99851483),
   ('independenceday', 0.99851483),
   ('july4th', 0.99851483),
   ('america', 0.99851483),
   ('flag', 0.99851483),
   ('bundting', 0.99851483)]),
 (774,
  [('bomk', 0.50299728),
   ('book', 0.50299728),
   ('livre', 0.50299728),
   ('interview', 0.50299728),
   ('ekosystemorg', 0.50299728),
   ('ekosystem', 0.50299728),
   ('vitostreet', 0.50299728),
   ('edi

In [ ]:
# entrenamos un modelo binario con randomforest para cada categoria de imagen
model = RandomForest.trainClassifier(sc.parallelize(data), 2, {}, 3, seed=42)




In [21]:
tag_as_label = inference_images.flatMap(merge_tag_as_label).filter(lambda x:x[1][1]>0.2).aggregateByKey(list(),(lambda x,y: x+list((y,))),(lambda i,j: i+j)).filter(lambda x:len(x[1])>1).collect()

tag_as_label

[('sodiumhydroxide', [(971, 0.43217739), (781, 0.37137467)]),
 ('doily', [(836, 0.23864704), (815, 0.23522472)]),
 ('soe', [(220, 0.44202313), (545, 0.74752247)]),
 ('giant', [(836, 0.23864704), (815, 0.23522472)]),
 ('sky',
  [(503, 0.93601441),
   (367, 0.77648342),
   (545, 0.74752247),
   (596, 0.86975491),
   (245, 0.41936967)]),
 ('texture', [(938, 0.20166412), (647, 0.43266514)]),
 ('black',
  [(593, 0.99846905),
   (415, 0.22868274),
   (330, 0.21981542),
   (245, 0.41936967)]),
 ('tweed', [(836, 0.23864704), (815, 0.23522472)]),
 ('me', [(967, 0.33025736), (971, 0.43217739), (781, 0.37137467)]),
 ('blanket', [(836, 0.23864704), (815, 0.23522472)]),
 ('closeup', [(224, 0.29146379), (745, 0.76611984)]),
 ('uyma', [(972, 0.34671441), (751, 0.3281346)]),
 ('toronto', [(967, 0.33025736), (220, 0.46368244)]),
 ('\xc3\xa7i\xc3\xa7ek', [(415, 0.22868274), (330, 0.21981542)]),
 ('vivitar', [(971, 0.43217739), (781, 0.37137467)]),
 ('causticsoda', [(971, 0.43217739), (781, 0.37137467)])